**Función:** grid search y entrenamiento de arquitecturas CNN alternativas, con el dataset 10k desbalanceado.

**Requiere:**
- ../../datasets/DATASET10K/ (con una carpeta por label y las fotos sin recortar)

**Crea:** Fichero con el modelo entrenado (epoch con menor val_loss).
- modelos/tf_model_\<run_id\>

In [1]:
ejecutadoEnGoogleDrive = True # True si se ejecuta en Google Drive

In [2]:
if ejecutadoEnGoogleDrive:
    from google.colab import drive
    drive.mount('/content/drive')
    PATH = '/content/drive/clasificacion_imagenes/'
    ruta_dataset = '/content/drive/clasificacion_imagenes/datasets/DATASET10K/'
else:
    PATH = '../../1_Setup_MLOps'
    ruta_dataset = '../../datasets/DATASET10K/'

%cd {PATH}

/Projetos_dev/projetos/clasificacion_imagenes/1_Setup_MLOps


In [3]:
# !pip install mlflow
import mlflow
from tensorflow.random import set_seed
from numpy.random import seed
from tensorflow import keras
from tensorflow.keras.utils import image_dataset_from_directory
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, Conv2D, MaxPooling2D
from keras.models import Sequential
#from keras.optimizers import Nadam
from keras.callbacks import EarlyStopping
from keras.layers import Rescaling
from tensorflow.data import AUTOTUNE
from tensorflow import one_hot
from keras import backend as K
import pickle

In [5]:
# definir parámetros para grid search
RUN = [{'param_A':10, 'param_B':[(5,5), (3,3)], 'batch_size':64},  # RUN 1
       {'param_A':10, 'param_B':[(5,5), (3,3)], 'batch_size':128}, # RUN 2
       {'param_A':10, 'param_B':[(5,5), (3,3)], 'batch_size':256}, # RUN 3
       {'param_A':10, 'param_B':[(3,3), (3,3)], 'batch_size':128}, # RUN 4
       {'param_A':10, 'param_B':[(3,3), (3,3)], 'batch_size':256}, # RUN 5
       {'param_A':20, 'param_B':[(5,5), (5,5)], 'batch_size':64},  # RUN 6
       {'param_A':20, 'param_B':[(5,5), (5,5)], 'batch_size':128}, # RUN 7
       {'param_A':20, 'param_B':[(5,5), (5,5)], 'batch_size':256}, # RUN 8
       {'param_A':20, 'param_B':[(5,5), (3,3)], 'batch_size':64},  # RUN 9
       {'param_A':20, 'param_B':[(5,5), (3,3)], 'batch_size':128}, # RUN 10
       {'param_A':20, 'param_B':[(5,5), (3,3)], 'batch_size':256}, # RUN 11
       {'param_A':20, 'param_B':[(3,3), (3,3)], 'batch_size':64},  # RUN 12
       {'param_A':20, 'param_B':[(3,3), (3,3)], 'batch_size':128}, # RUN 13
       {'param_A':20, 'param_B':[(3,3), (3,3)], 'batch_size':256}, # RUN 14
       {'param_A':10, 'param_B':[(5,5), (5,5)], 'batch_size':64},  # RUN 15
       {'param_A':10, 'param_B':[(5,5), (5,5)], 'batch_size':128}, # RUN 16
       {'param_A':10, 'param_B':[(5,5), (5,5)], 'batch_size':256}, # RUN 17
# ...
       {'param_A':10, 'param_B':[(3,3), (3,3)], 'batch_size':64}]  # RUN 30

### Instrucciones

Por limitaciones de tiempo en Colab no se encadenan los runs (ciclo for, p. ej.)

Cuando se termina un run, reiniciar kernel e incrementar manualmente el valor de RUN_NUMERO

Si un run da error, ejecutar la última celda del notebook para cerrar el run en MLflow

In [6]:
RUN_NUMERO = 1 # cuando se termina y vuelve a abrir, se cambia a 2, después 3

In [7]:
params = RUN[RUN_NUMERO-1]

In [8]:
print('En este run se van a usar estos parámetros: {}'.format(params))

En este run se van a usar estos parámetros: {'param_A': 10, 'param_B': [(5, 5), (3, 3)], 'batch_size': 64}


In [9]:
IMG_SIZE = 224
epochs = 100 # máximo de epochs (si no dispara el early stopping)
validation_split = 0.2
# Fijar random state para obtener resultados reproductibles
seed(42) # So long and thanks for all the fish
set_seed(42)

In [ ]:
# Definir quién está ejecutando (para no machacar el MLflow de los compañeros)
# Descomentar la linea que aplique
# yo = 'Pedro_T'
# yo = 'Toni_V'
# yo = 'Carlos_H'
# yo = 'Carlos_C'
print('Notebook ejecutado por {}'.format(yo))

# Definir experimento y run (MLflow)

In [11]:
# Este run pertenence al experimento 4, que tiene el id "3" (string)
experiment_id = "3"
mlflow.set_experiment(experiment_id = experiment_id) 
run = mlflow.start_run( # crear run en MLflow
    experiment_id=experiment_id,
    tags={"Autor": yo, "params":params}
)
# activar auto logging
mlflow.tensorflow.autolog()
print('Vamos empezar el run {} en el experimento {}. Este run está {}.'.format(run.info.run_id, experiment_id, run.info.status))

Vamos empezar el run f8bc25969081441881710bf5ec8ba2cc en el experimento 3. Este run está RUNNING.


# Definir carga del dataset (con prefetch) y preprocesado

In [13]:
train_ds = image_dataset_from_directory(
  ruta_dataset,
  validation_split=validation_split,
  subset="training",
  seed=42,
  image_size=(IMG_SIZE,IMG_SIZE),
  batch_size=params['batch_size'])\
.cache()\
.prefetch(buffer_size=AUTOTUNE).map(lambda x, y:
                                    (Rescaling(1./255)(x),
                                     one_hot(y, 5)))

Found 8312 files belonging to 5 classes.
Using 6650 files for training.


In [14]:
test_ds = image_dataset_from_directory(
  ruta_dataset,
  validation_split=validation_split,
  subset="validation",
  seed=42,
  image_size=(IMG_SIZE,IMG_SIZE),
  batch_size=params['batch_size'])\
.cache()\
.prefetch(buffer_size=AUTOTUNE).map(lambda x, y:
                                    (Rescaling(1./255)(x),
                                     one_hot(y, 5)))

Found 8312 files belonging to 5 classes.
Using 1662 files for validation.


In [15]:
categories = ['drink', 'food', 'inside', 'menu', 'outside'] # train_ds.class_names
num_labels = len(categories)

In [16]:
model = Sequential()
model.add(Conv2D(params['param_A'],
                 (12, 12),
                 padding='same',
                 input_shape=(IMG_SIZE,IMG_SIZE,3)))
model.add(Activation('relu'))
model.add(Conv2D(params['param_A']+10,
                 (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=params['param_B'][0]))
model.add(Dropout(0.25))

model.add(Conv2D(params['param_A']+20,
                 (3, 3),
                 padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(params['param_A']+30,
                 (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=params['param_B'][1]))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
opt = keras.optimizers.Nadam(learning_rate=0.0001, schedule_decay=1e-6)

In [17]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [18]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['acc', f1_m, precision_m, recall_m])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 10)      4330      
                                                                 
 activation (Activation)     (None, 224, 224, 10)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 222, 222, 20)      1820      
                                                                 
 activation_1 (Activation)   (None, 222, 222, 20)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 44, 44, 20)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 44, 44, 20)        0         
                                                        

In [20]:
callback = EarlyStopping(monitor='val_loss',
                         patience=10,
                         restore_best_weights=True)

In [ ]:
history = model.fit(train_ds,
                    validation_data=test_ds,
                    epochs = epochs,
                    batch_size=params['batch_size'],
                    callbacks=[callback],
                    verbose=1)

Epoch 1/100
104/104 [==============================] - 3122s 28s/step - loss: 0.9884 - acc: 0.6353 - f1_m: 0.5968 - precision_m: 0.7109 - recall_m: 0.5203 - val_loss: 0.8806 - val_acc: 0.7551 - val_f1_m: 0.5846 - val_precision_m: 0.9180 - val_recall_m: 0.4302
Epoch 2/100
104/104 [==============================] - 11s 101ms/step - loss: 0.7671 - acc: 0.7391 - f1_m: 0.7222 - precision_m: 0.7964 - recall_m: 0.6626 - val_loss: 0.8330 - val_acc: 0.7605 - val_f1_m: 0.6583 - val_precision_m: 0.8948 - val_recall_m: 0.5228
Epoch 3/100
104/104 [==============================] - 11s 104ms/step - loss: 0.6972 - acc: 0.7650 - f1_m: 0.7580 - precision_m: 0.8154 - recall_m: 0.7094 - val_loss: 0.8161 - val_acc: 0.7515 - val_f1_m: 0.6770 - val_precision_m: 0.8751 - val_recall_m: 0.5542
Epoch 4/100
104/104 [==============================] - 11s 102ms/step - loss: 0.6622 - acc: 0.7728 - f1_m: 0.7653 - precision_m: 0.8204 - recall_m: 0.7182 - val_loss: 0.8114 - val_acc: 0.7413 - val_f1_m: 0.6848 - val_pre

# Guardar modelo entrenado

In [ ]:
model.save('../4_Modelacion/Experimento_4/modelos/tf_model_exp1_{}'.format(run.info.run_id))

In [ ]:
mlflow.end_run()